In [1]:
!pip3 install gensim
!pip3 install fuzzywuzzy
!pip3 install textblob

    100% |████████████████████████████████| 23.6MB 1.4MB/s 
    100% |████████████████████████████████| 1.4MB 14.9MB/s 
    100% |████████████████████████████████| 133kB 26.0MB/s 
    100% |████████████████████████████████| 61kB 19.8MB/s 
    100% |████████████████████████████████| 5.1MB 6.1MB/s 
    100% |████████████████████████████████| 552kB 23.7MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
    100% |████████████████████████████████| 645kB 20.1MB/s 


In [2]:
import os
from gensim.similarities import WmdSimilarity
import pandas as pd
import numpy as np
import gensim

from gensim import corpora
import gensim.downloader as api
from gensim.matutils import softcossim
from gensim.models import Word2Vec
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm_notebook
from nltk import word_tokenize
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score
import xgboost as xgb
from textblob import Word
from textblob import TextBlob
import spacy

!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')  

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# **Loading data**

In [0]:
train_path = os.path.join('drive','My Drive','quora','train_data_v3_processed.csv')
train_data = pd.read_csv(train_path)
train_data = train_data.dropna(how="any").reset_index(drop=True)

In [4]:
train_data.head()

,id,question1,question2,is_duplicate,question1_lemma,question1_tag,question2_lemma,question2_tag
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what be the step by step guide to invest in sh...,WP VBZ DT NN IN NN NN TO VB IN NN NN IN NN .,what be the step by step guide to invest in sh...,WP VBZ DT NN IN NN NN TO VB IN NN NN .
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,what be the story of kohinoor ( koh - i - noor...,WP VBZ DT NN IN NNP -LRB- NNP HYPH NNP HYPH NN...,what would happen if the indian government ste...,WP MD VB IN DT JJ NN VBD DT NNP -LRB- NNP HYPH...
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,how can -PRON- increase the speed of -PRON- in...,WRB MD PRP VB DT NN IN PRP$ NN NN IN VBG DT NNP .,how can internet speed be increase by hack thr...,WRB MD VB NN VB VBN IN VBG IN NN .
3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,why be -PRON- mentally very lonely ? how can -...,WRB VBP PRP RB RB JJ . WRB MD PRP VB PRP .,find the remainder when [ math]23^{24}[/math ]...,VB DT NN WRB -LRB- NN -RRB- VBZ VBN IN CD .
4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"which one dissolve in water quikly sugar , sal...","WDT CD NN IN NN RB NN , NN , NN CC NN FW NN .",which fish would survive in salt water ?,WDT NN MD VB IN NN NN .


# Constants

In [0]:
QUESTION1 = 'question1_lemma'
QUESTION2 = 'question2_lemma'

In [0]:
## TODO DELETE
#train_data['question1_lemma'] = train_data['question1']
#train_data['question2_lemma'] = train_data['question2']
#train_data = train_data[:10]

# **Fuzzy features**

In [0]:
def setFuzzyFeatures(df):  
  df['fuzz_ratio'] = df.apply(lambda x: fuzz.ratio(str(x[QUESTION1]), str(x[QUESTION2])), axis=1)
  df['fuzz_partial_ratio'] = df.apply(lambda x: fuzz.partial_ratio(str(x[QUESTION1]), str(x[QUESTION2])), axis=1)
  df['fuzz_partial_token_set_ratio'] = df.apply(lambda x: fuzz.partial_token_set_ratio(str(x[QUESTION1]), str(x[QUESTION2])), axis=1)
  df['fuzz_partial_token_sort_ratio'] = df.apply(lambda x: fuzz.partial_token_sort_ratio(str(x[QUESTION1]), str(x[QUESTION2])), axis=1)
  df['fuzz_token_set_ratio'] = df.apply(lambda x: fuzz.token_set_ratio(str(x[QUESTION1]), str(x[QUESTION2])), axis=1)
  df['fuzz_token_sort_ratio'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x[QUESTION1]), str(x[QUESTION2])), axis=1)

# **String features**

In [0]:
def _avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

def _numStopWords(sentence):
  return len([x for x in sentence.split() if x in stop_words])

def _numNumbers(sentence):
  return len([x for x in sentence.split() if x.isdigit()])

def _numUppercaseWords(sentence):
  return len([x for x in sentence.split() if x.isupper()])

def setStringFeatures(df):  
  df['len_q1'] = df.question1.apply(lambda x: len(str(x)))
  df['len_q2'] = df.question2.apply(lambda x: len(str(x)))
  df['diff_len'] = df.len_q1 - df.len_q2
  df['len_char_q1'] = df.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
  df['len_char_q2'] = df.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
  df['len_word_q1'] = df.question1.apply(lambda x: len(str(x).split()))
  df['len_word_q2'] = df.question2.apply(lambda x: len(str(x).split()))
  df['common_words'] = df.apply(lambda x: len(set(str(x[QUESTION1]).lower().split()).intersection(set(str(x[QUESTION2]).lower().split()))), axis=1)
  df['avg_word_q1'] = df.question1.apply(lambda x: _avg_word(x))
  df['avg_word_q2'] = df.question2.apply(lambda x: _avg_word(x))
  df['num_stop_words_q1'] = df.question1.apply(lambda x: _numStopWords(x))
  df['num_stop_words_q2'] = df.question2.apply(lambda x: _numStopWords(x))
  df['numerics_q1'] = df.question1.apply(lambda x: _numNumbers(x))
  df['numerics_q2'] = df.question2.apply(lambda x: _numNumbers(x))
  df['uppercase_q1'] = df.question1.apply(lambda x: _numUppercaseWords(x))
  df['uppercase_q2'] = df.question2.apply(lambda x: _numUppercaseWords(x))
  

# **Word2Vec features**

In [0]:
w2v_model = api.load("word2vec-google-news-300")

In [0]:
def _getDictionary(df):  
  documents = list(df[QUESTION1].apply(lambda x: x.split()))+list(df[QUESTION2].apply(lambda x: x.split()))
  
  return corpora.Dictionary(documents)
   
def _softCossim(row, dictionary, similarity_matrix):
  q1 = row[QUESTION1].split()
  q2 = row[QUESTION2].split()
  
  q1 = dictionary.doc2bow(q1)
  q2 = dictionary.doc2bow(q2)
  
  return softcossim(q1, q2, similarity_matrix)
 
def setWord2VecFeatures(df):
  dictionary = _getDictionary(df)
  similarity_matrix = w2v_model.similarity_matrix(dictionary)
  
  df['softcossim'] = df.apply(lambda row: _softCossim(row, dictionary, similarity_matrix), axis=1)
  

# **TextBlob**

In [0]:
def _numSpellingMistakes(sentence):
  mistakes = 0
  for word in sentence.split():
    if TextBlob(word).correct() != word:
      mistakes += 1
  return mistakes


def setTextBlobFeatures(df):
  df['mistakes_q1'] = df.question1.apply(lambda x: _numSpellingMistakes(x))
  df['mistakes_q2'] = df.question2.apply(lambda x: _numSpellingMistakes(x))

# **Spacy**

In [0]:
def _getSpacySimilarity(row):
  q1 = row[QUESTION1]
  q2 = row[QUESTION2]
  
  tokens1 = nlp(Q1)
  tokens2 = nlp(Q2)
  
  return tokens1.similarity(tokens2)

def setSpacyFeatures(df):
  df['spacy_sim'] = df.apply(lambda row: _getSpacySimilarity(row), axis=1)
  

# Merge Features **Functions**

In [0]:
def setFeatures(df):
  setFuzzyFeatures(df)
  setStringFeatures(df)
  setWord2VecFeatures(df)
  setTextBlobFeatures(df)
  setSpacyFeatures(df)

# **Execution**

In [0]:
df = train_data#.copy()

In [0]:
setFeatures(df)

In [0]:
df.head()